In [ ]:
!pip install transformers


In [ ]:
!pip install sacremoses

In [ ]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

# load model
en_to_fr_model_name = "Helsinki-NLP/opus-mt-en-fr"
fr_to_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
tokenizer_en_fr = MarianTokenizer.from_pretrained(en_to_fr_model_name)
model_en_fr = MarianMTModel.from_pretrained(en_to_fr_model_name)
tokenizer_fr_en = MarianTokenizer.from_pretrained(fr_to_en_model_name)
model_fr_en = MarianMTModel.from_pretrained(fr_to_en_model_name)

# define traslator
def translate(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(**inputs)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# load data
data = pd.read_csv('https://raw.githubusercontent.com/YUEWU1233/ML_French_Blancpain/main/training_data.csv')

# New DataFrame
new_data = pd.DataFrame(columns=['id', 'sentence', 'difficulty'])

# back translation
for index, row in data.iterrows():
    # To English
    translated_to_en = translate(row['sentence'], model_fr_en, tokenizer_fr_en)[0]
    # To French
    back_translated_to_fr = translate(translated_to_en, model_en_fr, tokenizer_en_fr)[0]
    # Add to new dataset
    new_data.loc[index] = [row['id'], back_translated_to_fr, row['difficulty']]

    if (index + 1) % 50 == 0:
        print(f"already print {index + 1} records")

# save to new dataset
new_data.to_csv('back_translation.csv', index=False)

print("done!")

already print 50 records
already print 100 records
already print 150 records
already print 200 records
already print 250 records
already print 300 records
already print 350 records
already print 400 records
already print 450 records
already print 500 records
already print 550 records
already print 600 records
already print 650 records
already print 700 records
already print 750 records
already print 800 records
already print 850 records
already print 900 records
already print 950 records
already print 1000 records
already print 1050 records
already print 1100 records
already print 1150 records
already print 1200 records
already print 1250 records
already print 1300 records
already print 1350 records
already print 1400 records
already print 1450 records
already print 1500 records
already print 1550 records
already print 1600 records
already print 1650 records
already print 1700 records
already print 1750 records
already print 1800 records
already print 1850 records
already print 1900 re

In [ ]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.2 MB/s eta 0:00:00


In [ ]:
# Data Augmentation
import nlpaug.augmenter.word as naw

def augment_text(dataframe, augmenter, times=1):
    augmented_sentences = []
    for _, row in dataframe.iterrows():
        for _ in range(times):
            augmented_sentence = augmenter.augment(row['sentence'])
            augmented_sentences.append({'sentence': augmented_sentence, 'difficulty': row['difficulty']})
    return pd.DataFrame(augmented_sentences)

# Apply augmentation
text_augmenter = naw.SynonymAug(aug_src='wordnet')
augmented_data = augment_text(data, text_augmenter, times=1)
augmented_data.to_csv('nlpaug.csv', index=False)